In [14]:
import requests
from bs4 import BeautifulSoup

import csv

Скачивание данных с прямого эфира Яндекса

In [15]:
data_set_size = 10000

In [16]:
REQUESTS = set()

In [17]:
while len(REQUESTS) < data_set_size:
    res = requests.get('https://export.yandex.ru/last/last20x.xml')
    ans = BeautifulSoup(res.text, 'html.parser')
    
    for l in ans.findAll(name='item'):
        REQUESTS.add(l.get_text())

In [23]:
REQUESTS = list(REQUESTS)[:10000]

In [24]:
csvfile = 'search_queries.csv'

with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in REQUESTS:
        writer.writerow([val]) 

Кластеризация данных

In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pymorphy2
from string import digits

Предварительно подготовим данные, проведя лемматизацию - приведение словоформы к нормальной форме и убрав цифры

In [42]:
m = pymorphy2.MorphAnalyzer()

In [43]:
norm_form_requests = [' '.join([m.parse(word)[0].normal_form for word in x.split()]) for x in REQUESTS]

In [99]:
data_requests = [''.join(i for i in x if not i.isdigit()) for x in norm_form_requests]

Преобразуем каждый запрос в числовой вектор, используя метод TF-IDF, установив порог max_df = 0.9 (10% наиболее частых слов будут отброшены, для устранения предлогов, которые не несут смысловой информации)

In [101]:
vect = TfidfVectorizer(max_df= 0.9, min_df= 2)
vectoriz = vect.fit_transform(data_requests)
vectoriz.shape

data = vectoriz.toarray()

K-means

In [103]:
from sklearn.cluster import KMeans

In [109]:
k_means = KMeans(n_clusters=6, init='k-means++', max_iter=300)

k_means.fit(data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [110]:
dct = {}
for key, label in zip(REQUESTS, k_means.labels_):
    dct[label] = dct.get(label, []) + [key]  

In [112]:
dct[4]

['примеры  штанг для белья',
 'гороскоп для раков на ноябрь 2015 года для мужчин',
 'мкб 10 скачать',
 'обзор мфу для офиса 2015',
 'драйвер для ноутбука acer камера',
 'как выбрать масло для автомобиля',
 'ребенок и его права консультация для родителей',
 'а п чехов презентация для начальной школы',
 'clipson.ru скачать клипы бесплатно и без регистрации',
 'подработка для женщин после 50 лет в москве вакансии',
 'как устроен кулер для воды схема',
 'настольные игры для мальчиков',
 'альфамед все для стоматологии прайс',
 'ночник-проектор для детей',
 'скачать прошивку для samsung galaxy s3',
 'игры снайперы скачать',
 'натали как жаль скачать бесплатно',
 'гоголь мёртвые души скачать fb2',
 'оснастки для печатей и штампов',
 'презентация про химический элемент железо скачать',
 'jonas nik скачать бесплатно mp3',
 'самовсасывающий насос для дизельного топлива 220в',
 'комната для',
 'Порошок для ПММ "LOTTA" Allin1 MEGA PACK отзывы',
 'запчасти для ноутбуков в новосибирске',
 'мод для п

In [60]:
print(vectoriz[0])

  (0, 2383)	0.5985238335761836
  (0, 3051)	0.5529934411921472
  (0, 3375)	0.5796270133799287


In [65]:
vect.get_feature_names()[3375]

'соревнование'

In [67]:
data[0][3375]

0.5796270133799287

In [102]:
vectoriz.shape

(10000, 4003)